In [27]:
import os
import cv2
import tkinter
import matplotlib
import numpy as np
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib tk

In [28]:
def pointLoad(imgpath,pts):
    if os.path.exists(imgpath):
        img = cv2.imread(imgpath)
        plt.imshow(img)
        ptMatrix = plt.ginput(pts)
        # plt.close()
        return img, np.array(ptMatrix).astype(int)
        

    else:
        print("Invalid image path")
        return

In [29]:
def getH(dest, src):
    p = np.array([0,0,0,0,0,0,0,0,1])
    # if len(dest) < 5 and len(src) < 5:
        

    for i in range(len(dest)):
        pi = np.array([[-src[i][0],-src[i][1],-1,0,0,0,src[i][0]*dest[i][0],src[i][1]*dest[i][0],dest[i][0]],
        [0,0,0,-src[i][0],-src[i][1],-1,src[i][0]*dest[i][1],src[i][1]*dest[i][1],dest[i][1]]])
        p = np.vstack((pi,p))

  
    x = np.zeros((len(dest)*2,1))
    x = np.vstack((x,np.array([1])))
    H, residuals, rank, s = np.linalg.lstsq(p,x)
    H = H.reshape((3,3))
    

    
    return H
        


In [30]:
def pano(image_paths):
    imgs = []
    imgs_gray = []
    
    orbvals={"keypts":[],"descrpt":[],"matches":[]}
    orb = cv2.ORB_create()
    # sift = cv2.xfeatures2d.SIFT_create(nfeatures=200)
    for i in range(len(image_paths)):
        imgs.append(cv2.imread(image_paths[i]))
        imgs[i]=cv2.resize(imgs[i],(0,0),fx=0.4,fy=0.4)
        imgs_gray.append(cv2.cvtColor(imgs[i],cv2.COLOR_BGR2GRAY))
        keypt, desc = orb.detectAndCompute(imgs_gray[i],None)
        # keypt, desc = sift.detectAndCompute(imgs_gray[i], None)
        orbvals["keypts"].append(keypt)
        orbvals["descrpt"].append(desc)
        # print((orbvals["keypts"]))
        # cv2.imshow("image",cv2.drawKeypoints(imgs[i], orbvals["keypts"][i], None, (255, 0, 255)))


    bf = cv2.BFMatcher_create(cv2.NORM_HAMMING)    
    for i in range(len(imgs)-1):
        orbvals["matches"].append(bf.knnMatch(orbvals["descrpt"][i], orbvals["descrpt"][i+1],k=2))
    print(orbvals['matches'])
    cv2.imshow(draw_matches(imgs[0],orbvals["keypts"][0],imgs[1],orbvals["keypts"][1],orbvals["matches"][0]))


In [31]:
def draw_matches(img1, keypoints1, img2, keypoints2, matches):
  r, c = img1.shape[:2]
  r1, c1 = img2.shape[:2]

  # Create a blank image with the size of the first image + second image
  output_img = np.zeros((max([r, r1]), c+c1, 9), dtype='uint8')
  output_img[:r, :c, :] = np.dstack([img1, img1, img1])
  output_img[:r1, c:c+c1, :] = np.dstack([img2, img2, img2])

  # Go over all of the matching points and extract them
  for match in matches:
    (x1, y1) = keypoints1[img1_idx].pt
    (x2, y2) = keypoints2[img2_idx].pt

    # Draw circles on the keypoints
    cv2.circle(output_img, (int(x1),int(y1)), 4, (0, 255, 255), 1)
    cv2.circle(output_img, (int(x2)+c,int(y2)), 4, (0, 255, 255), 1)

    # Connect the same keypoints
    cv2.line(output_img, (int(x1),int(y1)), (int(x2)+c,int(y2)), (0, 255, 255), 1)
    
  return output_img

In [22]:
img1, img1pts = pointLoad("image1.jpg",8)

img2, img2pts = pointLoad("image2.jpg",8)




In [ ]:

cv2.imshow("image 1",img1)
cv2.imshow("image 2",img2)


In [23]:
H = getH(img1pts,img2pts)
im_out = cv2.warpPerspective(img2,H,(img1.shape[1],img1.shape[0]))
mask = np.zeros(img1.shape[:2], dtype="uint8")
cv2.fillConvexPoly(mask, np.int32([img1pts]),color=255)
cv2.fillConvexPoly(img1, np.int32([img1pts]),color=0)
masked =cv2.bitwise_and(im_out,im_out,mask=mask)
img3 = cv2.add(img1,masked)
plt.imshow(img3)

plt.show()

/var/folders/xh/b9gkq_p12x95yb6qg4603lq40000gn/T/ipykernel_26765/3460832648.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  H, residuals, rank, s = np.linalg.lstsq(p,x)


In [16]:

# image_paths=['s1.jpeg','s2.jpeg','s3.jpeg','s4.jpeg','s5.jpeg']
image_paths=['s1.jpeg','s2.jpeg']


# stitchy=cv2.Stitcher.create()
# (dummy,output)=stitchy.stitch(imgs)

# if dummy != cv2.STITCHER_OK:

# 	print("stitching ain't successful")
# else:
# 	print('Your Panorama is ready!!!')

pano(image_paths)


# final output
# cv2.imshow('final result',output)

# cv2.waitKey(0)


[((<DMatch 0x198ebc190>, <DMatch 0x198ebc030>), (<DMatch 0x198ebc110>, <DMatch 0x198ebc4b0>), (<DMatch 0x198ebc230>, <DMatch 0x198ebc270>), (<DMatch 0x198ebc1f0>, <DMatch 0x198e6f390>), (<DMatch 0x198e6f1d0>, <DMatch 0x198e6f430>), (<DMatch 0x198e6f650>, <DMatch 0x198e6f630>), (<DMatch 0x198e6f610>, <DMatch 0x198e6f2b0>), (<DMatch 0x198e6f5d0>, <DMatch 0x198e6f370>), (<DMatch 0x198e6f170>, <DMatch 0x198e6f3f0>), (<DMatch 0x198e6f270>, <DMatch 0x198e6f350>), (<DMatch 0x198e6f2d0>, <DMatch 0x198e6f310>), (<DMatch 0x198e6f590>, <DMatch 0x198e6f210>), (<DMatch 0x198e6f570>, <DMatch 0x198e6f550>), (<DMatch 0x198e6f530>, <DMatch 0x198e6f4b0>), (<DMatch 0x198e6f4d0>, <DMatch 0x198e6f4f0>), (<DMatch 0x198e6f6b0>, <DMatch 0x198e6f490>), (<DMatch 0x198e6f6d0>, <DMatch 0x198e6f6f0>), (<DMatch 0x198e6f710>, <DMatch 0x198e6f730>), (<DMatch 0x198e6f750>, <DMatch 0x198e6f770>), (<DMatch 0x198e6f790>, <DMatch 0x198e6f7b0>), (<DMatch 0x198e6f7d0>, <DMatch 0x198e6f7f0>), (<DMatch 0x198e6f810>, <DMatch 0

AttributeError: 'tuple' object has no attribute 'queryIdx'